In [1]:
import os
import scipy.io
from io import BytesIO
import base64
from PIL import Image, ImageDraw, ImageFont
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [2]:
TRAIN_HICO_IMAGE = '/data/dataset/HICO-DET/images/train2015/'
TEST_HICO_IMAGE = '/data/dataset/HICO-DET/images/test2015/'
ANNO_BBOX_MAT = '/data/dataset/HICO-DET/anno_bbox.mat'
ANNO_MAT = '/data/dataset/HICO-DET/anno.mat'

TRAIN_OUT_PATH = "outputs/hico-det_train.tsv"
TEST_OUT_PATH = "outputs/hico-det_test.tsv"

In [3]:
print(f'Train: {len(os.listdir(TRAIN_HICO_IMAGE))}, Test: {len(os.listdir(TEST_HICO_IMAGE))}')
anno_bbox = scipy.io.loadmat(ANNO_BBOX_MAT)
anno = scipy.io.loadmat(ANNO_MAT)
anno_bbox.keys(), anno.keys()

Train: 38118, Test: 9658


(dict_keys(['__header__', '__version__', '__globals__', 'bbox_train', 'bbox_test', 'list_action']),
 dict_keys(['__header__', '__version__', '__globals__', 'list_action', 'anno_train', 'anno_test', 'list_train', 'list_test']))

In [4]:
def to_tsv(mode='train'):
    targets = ""
    for j, bbox_train in enumerate(anno_bbox[f'bbox_{mode}'][0]):
        targets += f"{j+1}\t"
        
        hico_image = f"/data/dataset/HICO-DET/images/{mode}2015/"
        img_path = os.path.join(hico_image, bbox_train[0][0])
        img = Image.open(img_path)#.convert("RGB")
        img_buffer = BytesIO()
        img.save(img_buffer, format=img.format)
        byte_data = img_buffer.getvalue()
        base64_str = base64.b64encode(byte_data) # bytes
        base64_str = base64_str.decode("utf-8") # str
        targets += base64_str+'\t'

        for i, hoi in enumerate(bbox_train[2][0]):
            if len(hoi[1]) == 0:
                continue

            hoi_id = int(hoi[0])-1
            object_name = anno_bbox['list_action'][hoi_id][0][0][0]
            hoi_name = anno_bbox['list_action'][hoi_id][0][1][0]
            # print(hoi_id+1, object_name, hoi_name)
            bbox_human = hoi[1][:][0]
            bbox_object = hoi[2][:][0]

            for k, index in enumerate(hoi[3]):
                bbox_hu = bbox_human[int(index[0])-1]
                target = f"{int(bbox_hu[0])},{int(bbox_hu[2])},{int(bbox_hu[1])},{int(bbox_hu[3])},{hoi_id+1},{hoi_name},"
                
                bbox_ob = bbox_object[int(index[1])-1]
                target += f"{int(bbox_ob[0])},{int(bbox_ob[2])},{int(bbox_ob[1])},{int(bbox_ob[3])},{int(index[1])},{object_name}"
            if i != len(bbox_train[2][0])-1:
                target += "&&"
            targets += target
                
        targets += f'\n'

    out_path = f"outputs/hico-det_{mode}.tsv"
    with open(out_path, "w", encoding='utf-8') as f:
        f.write(targets)

In [5]:
# to_tsv('train')
to_tsv('test')